# Tutorial on Variational Quantum Eigensolver

---
This code was mostly based on Qiskit Variational Quantum Eigensolver Playground, you can find in [Github](https://github.com/JavaFXpert/vqe-playground). It was reduced as much as possible to analyze the core of the VQE Algorithm.  

In the coming paragraphs we are going to analyze all the code step by step. Please run all the cells in order.

To start we will be briefly explaining the algorithm and the technique behind the Variational Quantum Eigensolver (VQE). VQE is an hybrid quantum algorithm that leverages both classical and quantum computing. In the following figure you can see both elements of the algorithm clearely differentiated in Blue and Green.   
<img src="./VQE1.png" >

There are many ways to tune one or more parameters from the Quantum Circuit. In this example we will be finetuning the rotation angle of 25 different Rotation Gates through the Y axis. 
We will be finetuning the gates angle one by one, as a result you will see that we will run the optimization for at least 25 times, but the amount of gates to be tuned can be freely chosen as long as the rotations explore all the hilbert space of possible ansatz quantum states.



BlaBLA
DIAGRAMA
0 0 0 optimization circuit, - ansatz - hamiltonian  


### Definitions and qiskit versions  

We import all necessary functions from IBM qiskit and numpy python **libraries**, that we will later use on this Notebook.


In [15]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, BasicAer, execute
from qiskit.optimization.applications.ising import max_cut
from qiskit.aqua.operators.legacy import op_converter as op_c
from qiskit_textbook.tools import array_to_latex

print("Libraries loaded successfully.")

#import qiskit
#from qiskit import IBMQ
#from qiskit.visualization import plot_bloch_multivector, plot_histogram
#IBMQ.save_account('b34d54b460e56752c548f81ff431174ddb290d1572bd92a6e92e5a1a5afd830c2cccc4c806a4312f34f429eb372244dd2bea3e5360608e65584d1a9307938914')

Libraries loaded successfully.


### Definition of the Circuit
In these two clasess `CircuitGridNode` and `CircuitGridModel` we will be defining the Circuit that will be used to finetune the Ansatz State until we reach the ideal State that minimizes the energy of the Hamiltonian. The circuit will be modified with the angles on Rotate in Y axis Gate.

First Lets define the Class: **`CircuitGridNode`**.

In [16]:
class CircuitGridNode():
    ###################################################################
    # Structure of a node is a Gate with properties, 
    # ctrl_a points at the controlling wire of that Gate, -1 is no control
    ###################################################################
    def __init__(self, node_type, radians=0.0, ctrl_a=-1):
        self.node_type = node_type
        self.radians = radians
        self.ctrl_a = ctrl_a
        self.wire_num = -1
        self.column_num = -1


Now Lets define the Class: **`CircuitGridModel`**. Grid-based model of circuit, 2D ARRAY of nodes

Functions are:


* First Item  
  * BlaNested item 1
  - Bla Nested item 2
    
* Second Item
- First Item
- Second Item

In [17]:
class CircuitGridModel():

    def __init__(self, max_wires, max_columns):
        ###################################################################
        # Create a 5x21 array of empty nodes
        ###################################################################
        self.max_wires = max_wires
        self.max_columns = max_columns
        self.nodes = np.empty((max_wires, max_columns),
                                dtype = CircuitGridNode)

    def set_node(self, wire_num, column_num, circuit_grid_node):
        ###################################################################
        # put node in circuit
        ###################################################################
        # First, embed the wire and column locations in the node
        circuit_grid_node.wire_num = wire_num
        circuit_grid_node.column_num = column_num
        self.nodes[wire_num][column_num] = circuit_grid_node

    def get_rotation_gate_nodes(self):
        ###################################################################
        # get a list of nodes suitable for rotation (not controlled)
        # Rotar si es X, Y o Z
        # Don't allow rotation of CONTROLLED  X or Y gates
        # ctrl_a == -1 means que NO es CONTROLLED
        ###################################################################
        rot_gate_nodes = []
        for column_num in range(self.max_columns):
            for wire_num in range(self.max_wires):
                node = self.nodes[wire_num][column_num]
                if node and node.node_type == Y:
                    rot_gate_nodes.append(node)
        return rot_gate_nodes

    def compute_circuit_simple(self):
        ###################################################################
        # build the quantum circuit with quantum gates based on the model.
        # Transform the node 5x21 array into a QISKIT Circuit and return it
        # Use only control X and Rot Y
        ###################################################################
        qr = QuantumRegister(self.max_wires, 'q')
        qc = QuantumCircuit(qr)

        for column_num in range(self.max_columns):
            for wire_num in range(self.max_wires):
                node = self.nodes[wire_num][column_num]
                if node:
                    if node.node_type == X:     # Controlled X gate
                        qc.cx(qr[node.ctrl_a], qr[wire_num])
                    elif node.node_type == Y:   # Rotation around Y axis
                        qc.ry(node.radians, qr[wire_num])
        return qc

In [18]:
class ExpectationGrid():
        #########################################################################################
        #  This "grid" contains basis states and Hamiltonian eigenvalues
        #  and calculates expectation value based on it
        #########################################################################################
    def __init__(self, circuit, adj_matrix):
        #self.maxcut_to_matrix = None
        #self.matrixdiag = None
        self.eigenvalues = None
        self.maxcut_shift = 0
        self.basis_states = []
        self.quantum_state = None
        #########################################################################################
        #  basis_states es una array con los nombres de las bases eg |10010>       
        #########################################################################################
        for idx in range(2**NUM_QUBITS):
            state = format(idx, '0' + str(NUM_QUBITS) + 'b')
            self.basis_states.append(state)

        self.set_adj_matrix(adj_matrix)
        self.set_circuit(circuit)

    def set_circuit(self, circuit):
        backend_sv_sim = BasicAer.get_backend('statevector_simulator')
        job_sim = execute(circuit, backend_sv_sim)
        result_sim = job_sim.result()
        self.quantum_state = result_sim.get_statevector(circuit, decimals=3)

    def set_adj_matrix(self, adj_matrix):
        #########################################################################################
        # THIS OUTPUTS Eigenvalues of the Hamiltonian.
        # La funcion Max cut crea un operador Hermitian (Hamiltonian) basado en el entry de un adjacency matrix
        # Should expand for other optimization algorithm
        # La clave es como pasar del problema de optimizacion al Hamiltonian (operator)
        # este es built-in de Qiskit pero estaria bien desarrollar uno ground-up
        #########################################################################################
        maxcut_op, self.maxcut_shift = max_cut.get_operator(adj_matrix)
        maxcut_to_matrix = op_c.to_matrix_operator(maxcut_op)
        self.eigenvalues = maxcut_to_matrix.dia_matrix
        
        #PRINT HAMILTONIAN OF ADJACENCY MATRIX
        print("Ising Hamiltonian of the Adjacency Matrix:")
        print(maxcut_op.print_details())
            
    def imprime_state(self, statevector_probs):
        state_str = ' Ansatz Quantum State = '
        for idx in range(len(statevector_probs)):
            if statevector_probs[idx] != 0:
                state_str += ("{:.2f}".format(statevector_probs[idx]) + "*|" + self.basis_states[idx] + "> + ")
        print (state_str[:-2])
    
    def calc_expectation_value(self):
        #########################################################################################
        # CALCULO DEL EXPECTATION VALUE del current Quantum State vs HAMILTONIAN
        # <Phi|H|Phi> = sum (eigen*|phi_i|^2) "Energia de ese State"
        #########################################################################################
        statevector_probs = np.absolute(self.quantum_state) ** 2      
        exp_val = np.sum(self.eigenvalues * statevector_probs)

        #########################################################################################
        # Indice del estado que mayor probabilidad tiene. Si hay mas de uno, devuelve solo el 1o
        #########################################################################################
        basis_state_idx = np.argmax(statevector_probs)
        self.imprime_state(statevector_probs)
        #########################################################################################
        # Devuelve el valor esperado del acual estado cuantico y
        # el indice de la componente ortogonal con mayor probabilidad.
        #########################################################################################
        return exp_val, self.basis_states[basis_state_idx]


In [19]:
def calc_new_energy(circuit_grid_model, expectation_grid, rotation_gate_nodes):
    #########################################################################################
    # - This function will CALL QUANTUM CIRCUIT (calc.expect.value)
    #   -set the optimized angles to the "rotable gates" in circuit
    #   -return the cost/distance/energy of the current state (in expectation grid) in Hamiltonian  
    #########################################################################################
    for idx in range(len(rotation_gate_nodes)):
        rotation_gate_nodes[idx].radians = optimized_rotations[idx]

    # Calculate resulting state with new circuit (optimized angles) 
    expectation_grid.set_circuit(circuit_grid_model.compute_circuit_simple())
    # Calculate Energy with new State
    distance, basis_state1 = expectation_grid.calc_expectation_value()
    return distance, basis_state1

def optimize_rotations(circuit_grid_model, expectation_grid, rotation_gate_nodes):
    global min_distance, move_radians, cur_rotation_num, rot_direction, basis_state_str, Fin_optimizacion

    # remember: optimized rotations are the rotations angle for each "rotatable" gate
    # cur_rotation_num es el Gate actual siendo optimizado
    
    if cur_rotation_num < len(optimized_rotations):   # Comprueba que no hemos llegado al ultimo gate optimizable
        
        cur_ang_rad = optimized_rotations[cur_rotation_num]
        proposed_cur_ang_rad = cur_ang_rad
        proposed_cur_ang_rad += move_radians * rot_direction
        
        if (0.0 <= proposed_cur_ang_rad <= np.pi * 2 + 0.01) and (num_times_rot_dir_change[cur_rotation_num]<2):
            
            optimized_rotations[cur_rotation_num] = proposed_cur_ang_rad   
            
            # Calcula nueva energia con nuevo ansatz 
            # CALL QUANTUM CIRCUIT HERE!!!!!!
            temp_distance, basis_state_str = calc_new_energy(circuit_grid_model, expectation_grid, rotation_gate_nodes)
            
            if temp_distance > min_distance: 
                # NOT OPTIMIZED. Distance is increasing so restore the angle in the array and change direction of rotation. + Increase Counter of drection change times
                optimized_rotations[cur_rotation_num] = cur_ang_rad
                rot_direction *= -1
                num_times_rot_dir_change[cur_rotation_num] += 1
            
            else:
                # OPTIMIZED or equal. Distance decreasing, so keep the proposed angle and update Energy
                min_distance = temp_distance

        else:                                       
            cur_rotation_num += 1              # Se salio del rango de 0 a 2pi. o cambio 2 veces de direccion de rotacion.
                                                    # Termina la rotacion este gate AND Move to Next GATE para Rotacion
    else:
        Fin_optimizacion = True    # hemos llegado al ultimo gate optimizable, se acabo el proceso de optimizacion
    
    return min_distance, basis_state_str


In [20]:
#################################################################################################
# Initializes Optimizacion variables, 1st CIRCUIT , and Expectation grid (hamiltonian based on adjacency matrix)
#################################################################################################
NUM_QUBITS = 5
NUM_STATE_DIMS = 2**NUM_QUBITS
X = 1 ; Y = 2 ; Z = 3           # Gate Definitions    
CTRL = 11                       # "control" part of multi-qubit gate
circuit_grid_model = None
expectation_grid = None
optimized_rotations = None
cur_rotation_num = 0
min_distance = 0
basis_state_str = ''
proposed_cur_ang_rad = 0
cur_ang_rad = 0
rot_direction = 1
move_radians = np.pi / 8     # Rotation Step

#################################################################################################
# Matriz de adyacencia
#################################################################################################
initial_adj_matrix = np.array([
    [0, 3, 1, 3, 0],
    [3, 0, 0, 0, 2],
    [1, 0, 0, 3, 0],
    [3, 0, 3, 0, 2],
    [0, 2, 0, 2, 0]
])

#################################################################################################
# MONTAR EL CIRCUITO INICIAL que pasa de |00000> al estado cuantico ansatz
#################################################################################################

circuit_grid_model = CircuitGridModel(NUM_QUBITS, 21)

circuit_grid_model.set_node(0, 0, CircuitGridNode(Y, np.pi))
circuit_grid_model.set_node(1, 0, CircuitGridNode(Y, np.pi))
circuit_grid_model.set_node(2, 0, CircuitGridNode(Y, np.pi))
circuit_grid_model.set_node(3, 0, CircuitGridNode(Y, np.pi))
circuit_grid_model.set_node(4, 0, CircuitGridNode(Y, np.pi))

circuit_grid_model.set_node(1, 1, CircuitGridNode(X, 0, 0))
circuit_grid_model.set_node(2, 2, CircuitGridNode(X, 0, 1))
circuit_grid_model.set_node(3, 3, CircuitGridNode(X, 0, 2))
circuit_grid_model.set_node(4, 4, CircuitGridNode(X, 0, 3))

circuit_grid_model.set_node(0, 5, CircuitGridNode(Y, np.pi))
circuit_grid_model.set_node(1, 5, CircuitGridNode(Y, np.pi))
circuit_grid_model.set_node(2, 5, CircuitGridNode(Y, np.pi))
circuit_grid_model.set_node(3, 5, CircuitGridNode(Y, np.pi))
circuit_grid_model.set_node(4, 5, CircuitGridNode(Y, np.pi))

circuit_grid_model.set_node(1, 6, CircuitGridNode(X, 0, 0))
circuit_grid_model.set_node(2, 7, CircuitGridNode(X, 0, 1))
circuit_grid_model.set_node(3, 8, CircuitGridNode(X, 0, 2))
circuit_grid_model.set_node(4, 9, CircuitGridNode(X, 0, 3))

circuit_grid_model.set_node(0, 10, CircuitGridNode(Y, np.pi))
circuit_grid_model.set_node(1, 10, CircuitGridNode(Y, np.pi))
circuit_grid_model.set_node(2, 10, CircuitGridNode(Y, np.pi))
circuit_grid_model.set_node(3, 10, CircuitGridNode(Y, np.pi))
circuit_grid_model.set_node(4, 10, CircuitGridNode(Y, np.pi))

circuit_grid_model.set_node(1, 11, CircuitGridNode(X, 0, 0))
circuit_grid_model.set_node(2, 12, CircuitGridNode(X, 0, 1))
circuit_grid_model.set_node(3, 13, CircuitGridNode(X, 0, 2))
circuit_grid_model.set_node(4, 14, CircuitGridNode(X, 0, 3))

circuit_grid_model.set_node(0, 15, CircuitGridNode(Y, np.pi))
circuit_grid_model.set_node(1, 15, CircuitGridNode(Y, np.pi))
circuit_grid_model.set_node(2, 15, CircuitGridNode(Y, np.pi))
circuit_grid_model.set_node(3, 15, CircuitGridNode(Y, np.pi))
circuit_grid_model.set_node(4, 15, CircuitGridNode(Y, np.pi))

circuit_grid_model.set_node(1, 16, CircuitGridNode(X, 0, 0))
circuit_grid_model.set_node(2, 17, CircuitGridNode(X, 0, 1))
circuit_grid_model.set_node(3, 18, CircuitGridNode(X, 0, 2))
circuit_grid_model.set_node(4, 19, CircuitGridNode(X, 0, 3))

circuit_grid_model.set_node(0, 20, CircuitGridNode(Y, np.pi))
circuit_grid_model.set_node(1, 20, CircuitGridNode(Y, np.pi))
circuit_grid_model.set_node(2, 20, CircuitGridNode(Y, np.pi))
circuit_grid_model.set_node(3, 20, CircuitGridNode(Y, np.pi))
circuit_grid_model.set_node(4, 20, CircuitGridNode(Y, np.pi))

circuit = circuit_grid_model.compute_circuit_simple()
expectation_grid = ExpectationGrid(circuit, initial_adj_matrix)

display(circuit.draw(output='text',scale=0.9,justify="left"))
array_to_latex(initial_adj_matrix, pretext="\\text{Adjacency Matrix = }\n")

############################Sacate el hamiltonian del adjancency matrix!


Ising Hamiltonian of the Adjacency Matrix:
IIIZZ	(1.5+0j)
IIZIZ	(0.5+0j)
IZIIZ	(1.5+0j)
IZZII	(1.5+0j)
ZIIZI	(1+0j)
ZZIII	(1+0j)



┌───────┐     ┌───────┐                  ┌───────┐                  »
q_0: ┤ RY(π) ├──■──┤ RY(π) ├─────────────■────┤ RY(π) ├─────────────■────»
     ├───────┤┌─┴─┐└───────┘┌───────┐  ┌─┴─┐  └───────┘┌───────┐  ┌─┴─┐  »
q_1: ┤ RY(π) ├┤ X ├────■────┤ RY(π) ├──┤ X ├──────■────┤ RY(π) ├──┤ X ├──»
     ├───────┤└───┘  ┌─┴─┐  └───────┘┌─┴───┴─┐  ┌─┴─┐  └───────┘┌─┴───┴─┐»
q_2: ┤ RY(π) ├───────┤ X ├──────■────┤ RY(π) ├──┤ X ├──────■────┤ RY(π) ├»
     ├───────┤       └───┘    ┌─┴─┐  └───────┘┌─┴───┴─┐  ┌─┴─┐  └───────┘»
q_3: ┤ RY(π) ├────────────────┤ X ├──────■────┤ RY(π) ├──┤ X ├──────■────»
     ├───────┤                └───┘    ┌─┴─┐  ├───────┤  └───┘    ┌─┴─┐  »
q_4: ┤ RY(π) ├─────────────────────────┤ X ├──┤ RY(π) ├───────────┤ X ├──»
     └───────┘                         └───┘  └───────┘           └───┘  »
«     ┌───────┐                  ┌───────┐                           
«q_0: ┤ RY(π) ├─────────────■────┤ RY(π) ├───────────────────────────
«     └───────┘┌───────┐  ┌─┴─┐  └───────┘┌───────┐                  
«q_1: ────■────┤ RY(π) ├──┤ X ├──────■────┤ RY(π) ├──────────────────
«       ┌─┴─┐  └───────┘┌─┴───┴─┐  ┌─┴─┐  └───────┘┌───────┐         
«q_2: ──┤ X ├──────■────┤ RY(π) ├──┤ X ├──────■────┤ RY(π) ├─────────
«     ┌─┴───┴─┐  ┌─┴─┐  └───────┘┌─┴───┴─┐  ┌─┴─┐  └───────┘┌───────┐
«q_3: ┤ RY(π) ├──┤ X ├──────■────┤ RY(π) ├──┤ X ├──────■────┤ RY(π) ├
«     ├───────┤  └───┘    ┌─┴─┐  ├───────┤  └───┘    ┌─┴─┐  ├───────┤
«q_4: ┤ RY(π) ├───────────┤ X ├──┤ RY(π) ├───────────┤ X ├──┤ RY(π) ├
«     └───────┘           └───┘  └───────┘           └───┘  └───────┘

<IPython.core.display.Math object>

# $ \langle\Psi|H|\Psi\rangle $

## Epigraph 2 What is this? This is it and no more
---

### Epigraph Level 2

bla bla   
vlla *vla* **vla** 
---
sure   

| Id | Label    | Price |
|--- |----------| ------|
| 01 | Markdown |\$1600 |
| 02 | is       |  \$12 |
| 03 | AWESOME  | \$999 |


**Input:** $ N$ ( $ N \ge 1 $ ) qubits in the $ |0 \dots 0\rangle $ state (stored in an array of length $ N $).   

To insert a mathematical formula we use the dollar symbol $, as follows:

Euler's identity: $ e^{i \pi} + 1 = 0 $


Qiskit provides a number of [Algorithms](https://qiskit.org/documentation/apidoc/qiskit.aqua.algorithms.html) and they are grouped by category according to the task they can perform. For instance `Minimum Eigensolvers` to find the smallest eigen value of an operator, for example ground state energy of a chemistry Hamiltonian or a solution to an optimization problem when expressed as an Ising Hamiltonian. There are `Classifiers` for machine learning  classification problems and `Amplitude Estimators` for value estimation that can be used say in financial applications. The full set of categories can be seen in the Algorithms documentation link above.

Algorithms are configurable and often part of the configuration will be in the form of smaller building blocks, of which different instances of the building block type can be given. For instance with `VQE`, the Variational Quantum Eigensolver, it takes a trial wavefunction, in the form of a `QuantumCircuit` and a classical optimizer among other things. Such building blocks can be found as [Components](https://qiskit.org/documentation/apidoc/qiskit.aqua.components.html) and as circuits from the [Circuit Library](https://qiskit.org/documentation/apidoc/circuit_library.html).

Let's take a look at an example to construct a VQE instance. Here `TwoLocal` is the variational form (trial wavefunction), a parameterized circuit which can be varied, and `SLSQP` a classical optimizer. These are created as separate instances and passed to VQE when it is constructed. Trying, for example, a different classical optimizer, or variational form is simply a case of creating an instance of the one you want and passing it into VQE.

In [21]:
# All optimized rotations intialized with rotation=Pi for all gates at the beginning
# Citcuit and Gates initially are indeed Pauli Y(PI) = Y
rotation_gate_nodes = circuit_grid_model.get_rotation_gate_nodes()
optimized_rotations = np.full(len(rotation_gate_nodes), np.pi)
num_times_rot_dir_change = np.zeros(len(optimized_rotations))

# 1st Optimization Iteration 
min_distance, basis_state_str = calc_new_energy(circuit_grid_model, expectation_grid, rotation_gate_nodes)
    
#################################################################################################
#  -  Main Loop
#################################################################################################
iteration=0
Fin_optimizacion = False        
while not Fin_optimizacion:
    #################################################################################################
    #  -  PRINTOUT de La optimizacion actualmente
    #################################################################################################
    if cur_rotation_num != len(optimized_rotations):
        print(' Iteration #' + str(iteration+1) , ', Rotating Gate #' , str(cur_rotation_num+1) , 
        '(Wire, Column) = (', rotation_gate_nodes[cur_rotation_num].wire_num+1, ',' , 
        rotation_gate_nodes[cur_rotation_num].column_num+1, '). Energy:', "{:.6f}".format(np.real(min_distance)), 
        ', Best Basis State:', basis_state_str, "\n")
        
    # Optimize
    min_distance, basis_state_str = optimize_rotations(circuit_grid_model, expectation_grid, rotation_gate_nodes)
    iteration +=  1

# FIN and EXIT + RESULTADO
print("\n################################################################################################# \n")
print(' min_distance: ', "{:.6f}".format(np.real(min_distance)), ', Optimal Basis State:', basis_state_str)
print("\n################################################################################################# \n")


 Ansatz Quantum State = 1.00*|00111> 
 Iteration #1 , Rotating Gate # 1 (Wire, Column) = ( 1 , 1 ). Energy: -1.000000 , Best Basis State: 00111 

 Ansatz Quantum State = 0.96*|00111> + 0.04*|10110> 
 Iteration #2 , Rotating Gate # 1 (Wire, Column) = ( 1 , 1 ). Energy: -1.038060 , Best Basis State: 00111 

 Ansatz Quantum State = 0.85*|00111> + 0.15*|10110> 
 Iteration #3 , Rotating Gate # 1 (Wire, Column) = ( 1 , 1 ). Energy: -1.146447 , Best Basis State: 00111 

 Ansatz Quantum State = 0.69*|00111> + 0.31*|10110> 
 Iteration #4 , Rotating Gate # 1 (Wire, Column) = ( 1 , 1 ). Energy: -1.308658 , Best Basis State: 00111 

 Ansatz Quantum State = 0.50*|00111> + 0.50*|10110> 
 Iteration #5 , Rotating Gate # 1 (Wire, Column) = ( 1 , 1 ). Energy: -1.500000 , Best Basis State: 00111 

 Ansatz Quantum State = 0.31*|00111> + 0.69*|10110> 
 Iteration #6 , Rotating Gate # 1 (Wire, Column) = ( 1 , 1 ). Energy: -1.691342 , Best Basis State: 10110 

 Ansatz Quantum State = 0.15*|00111> + 0.85*|1011

 Ansatz Quantum State = 0.96*|10101> + 0.04*|11111> 
 Iteration #59 , Rotating Gate # 12 (Wire, Column) = ( 2 , 11 ). Energy: -6.000000 , Best Basis State: 10101 

 Ansatz Quantum State = 0.96*|10101> + 0.04*|11111> 
 Iteration #60 , Rotating Gate # 12 (Wire, Column) = ( 2 , 11 ). Energy: -6.000000 , Best Basis State: 10101 

 Iteration #61 , Rotating Gate # 13 (Wire, Column) = ( 3 , 11 ). Energy: -6.000000 , Best Basis State: 10101 

 Ansatz Quantum State = 0.04*|00001> + 0.96*|10101> 
 Iteration #62 , Rotating Gate # 13 (Wire, Column) = ( 3 , 11 ). Energy: -6.000000 , Best Basis State: 10101 

 Ansatz Quantum State = 0.04*|00001> + 0.96*|10101> 
 Iteration #63 , Rotating Gate # 13 (Wire, Column) = ( 3 , 11 ). Energy: -6.000000 , Best Basis State: 10101 

 Iteration #64 , Rotating Gate # 14 (Wire, Column) = ( 4 , 11 ). Energy: -6.000000 , Best Basis State: 10101 

 Ansatz Quantum State = 0.96*|10101> + 0.04*|11101> 
 Iteration #65 , Rotating Gate # 14 (Wire, Column) = ( 4 , 11 ). Ener

---
This code was mostly based on Qiskit Variational Quantum Eigensolver Playground, you can find in [Github](https://github.com/JavaFXpert/vqe-playground). You can find the complete code with a fantastic visualization based on pygame.

---

## Keep this as reference until completed and then delete
---

### Epigraph Level 2

bla bla   
vlla *vla* **vla** 
---
sure   

| Id | Label    | Price |
|--- |----------| ------|
| 01 | Markdown |\$1600 |
| 02 | is       |  \$12 |
| 03 | AWESOME  | \$999 |


**Input:** $ N$ ( $ N \ge 1 $ ) qubits in the $ |0 \dots 0\rangle $ state (stored in an array of length $ N $).   

To insert a mathematical formula we use the dollar symbol $, as follows:

Euler's identity: $ e^{i \pi} + 1 = 0 $

Qiskit provides a number of [Algorithms](https://qiskit.org/documentation/apidoc/qiskit.aqua.algorithms.html) and they are grouped by category according to the task they can perform. For instance `Minimum Eigensolvers` to find the smallest eigen value of an operator, for example ground state energy of a chemistry Hamiltonian or a solution to an optimization problem when expressed as an Ising Hamiltonian. There are `Classifiers` for machine learning  classification problems and `Amplitude Estimators` for value estimation that can be used say in financial applications. The full set of categories can be seen in the Algorithms documentation link above.
